In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
from google.colab import drive

drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [3]:
df = pd.read_csv('gdrive/My Drive/reddit_worldnews_start_to_2016-11-22.csv')[:100000]

df.head()

,time_created,date_created,up_votes,down_votes,title,over_18,author,subreddit
0,1201232046,2008-01-25,3,0,Scores killed in Pakistan clashes,False,polar,worldnews
1,1201232075,2008-01-25,2,0,Japan resumes refuelling mission,False,polar,worldnews
2,1201232523,2008-01-25,3,0,US presses Egypt on Gaza border,False,polar,worldnews
3,1201233290,2008-01-25,1,0,Jump-start economy: Give health care to all,False,fadi420,worldnews
4,1201274720,2008-01-25,4,0,Council of Europe bashes EU&UN terror blacklist,False,mhermans,worldnews


In [0]:
from gensim.models import Word2Vec

text = [t.lower() for t in df.title]

model = Word2Vec(text, size=300, window=5, sg=1)

In [0]:
def doc_vectorizer(text, model):
  doc_vec = 0
  count = 0

  if len(text) == 1:
    return model[text]

  for t in text:
    try:
      word_vec = model[t]
      doc_vec = doc_vec + word_vec
      count += 1
    except:
      pass
  
  doc_vec = doc_vec / count
  return doc_vec

In [7]:
df['score'] = df.up_votes - df.down_votes

df_grouped = df.groupby('author')['title'].apply(lambda x: ' '.join(x)).reset_index()
df_grouped['mean_score'] = df.groupby('author')['score'].mean().values

df_grouped.head()

,author,title,mean_score
0,---sniff---,"The wife of Mir Hossein Mousavi, Ahmadinejad’s...",20.400000
1,--TacoLoco--,No explosives found in the bomb strapped to ...,5.000000
2,-____-,Palestinian sets record for longest imprisonme...,10.000000
3,-_o_0__0_o_-,Sweden wants to force ISPs to save user data S...,5.666667
4,-repick,Four Norwegian soldiers killed in Afghanistan,0.000000


In [8]:
author_vectors = [doc_vectorizer(t.lower(), model) for t in df_grouped.title]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # Remove the CWD from sys.path while we load stuff.


In [19]:
np.array(author_vectors).shape

(16268, 300)

In [13]:
#pip install doesn't work, so use conda
!pip install faiss-cpu --no-cache

     |████████████████████████████████| 7.2MB 2.7MB/s 


In [38]:
#https://github.com/facebookresearch/faiss/blob/master/tutorial/python/1-Flat.py

d = 300                           # dimension
nb = 10000                      # database size
nq = (len(author_vector) - 10000)                       # nb of queries

import faiss                   
index = faiss.IndexFlatL2(d)   # build the index
print(index.is_trained)

True


In [39]:
author_vector = np.array(author_vectors[:10000], dtype='float32')
query_vector = np.array(author_vectors[10000:], dtype='float32')
index.add(author_vector)                 # add vectors to the index
print(index.ntotal)

10000


In [41]:
k = 5                          # we want to see 4 nearest neighbors
D, I = index.search(author_vector[:5], k) # sanity check
print(I)
print(D)
D, I = index.search(query_vector, k)     # actual search
print(I)                   # neighbors of the 5 first queries

[[   0 2633 4722 4754 6902]
 [   1 1140  505  404    2]
 [   2 8927    1 8221 5940]
 [   3 2521 4326 9612 4788]
 [   4 3182 6299 4975 9726]]
[[0.         0.00014414 0.00015258 0.00016022 0.00018122]
 [0.         0.00079962 0.00080908 0.00085193 0.00088454]
 [0.         0.00082298 0.00088454 0.00091068 0.00091538]
 [0.         0.0009999  0.00101617 0.00105183 0.00109636]
 [0.         0.00177411 0.0018164  0.00184269 0.00187129]]
[[7648 2899  884 6945 9602]
 [1832 7918 3682 4713  192]
 [4597 3024 8359 7315 2294]
 ...
 [5385  271 1099 6022 4229]
 [8237 7259  907 2418 9108]
 [ 366 2570 2672 7893 1170]]
